# Continuous Control (for macbook pro m1)

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
# DONT CLOSE IT. let's reuse it for the next steps.
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import sys
import torch

# custom modules
from reacher.agent import Agent
from reacher.ddpg import DDPG

print("-> Python %s" % sys.version)
print("-> PyTorch %s" % torch.__version__)

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

Initialize our custom agent

In [ ]:
# Choose the fastest processor (in the hardware)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available() & torch.backends.mps.is_built():
    device = torch.device("mps")

# initialize the agent
agent: Agent = Agent(
    state_size = state_size, 
    action_size = action_size, 
    device = device
)

Now train the agent

In [ ]:
ddpg: DDPG = DDPG(env, brain_name, agent)
scores = ddpg.train()

Plot the scores of all the reachers

In [ ]:
# plt.figure()
# plt.plot(np.arange(len(scores)), scores)
# plt.title('Training history')
# plt.ylabel('Score')
# plt.xlabel('Episode')
# plt.legend(
#     [f"Reacher: {i}" for i in range(1, num_agents + 1)],
#     loc='center left',
#     bbox_to_anchor=(1, 0.5)
# )
# plt.savefig(f"assets/scores_{TIMESTAMP}.png", format="png", bbox_inches='tight')
# plt.show()

Plot the average scores of all reachers.

In [ ]:
# avg_scores = np.array(scores).mean(axis=1)
# avg_scores_window = [np.mean(avg_scores[max(0,i-100):i+1]) for i in range(len(avg_scores))]

# # plot the scores
# plt.figure()
# plt.plot(np.arange(1, len(scores)+1), avg_scores, label="window = 1")
# plt.plot(np.arange(1, len(scores)+1), avg_scores_window, label="window = 100")
# # plt.plot(np.arange(1, len(scores)+1), [30]*len(scores), linestyle='--', color='red')
# plt.hlines(y=30, xmin=0, xmax=200, colors='red', linestyles='dotted',label='goal')
# plt.title('Training history | Average Score')
# plt.ylabel('Score')
# plt.xlabel('Episode')
# plt.legend()
# plt.savefig(f"assets/scores_avg_{TIMESTAMP}.png", format="png", bbox_inches='tight')
# plt.show()

Save the (actor / critic) models 

In [ ]:
ddpg.save_models()

Define a new agent and only load the actor. We don't need any more critics ;) here.

In [ ]:
agent: Agent = Agent(
    state_size=state_size, 
    action_size=action_size, 
    seed=2, 
    device=device
)
agent.load_model('./models/actor.pth')

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states, add_noise=False)       # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()